In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LearningRateScheduler
from utils.preprocess import standard_color_aug
from utils.Load_ImageNet import load_training_images
from utils.AttentionResNet_ImageNet import AttentionResNet56
from utils.predict_10_crop import predict_10_crop, crop_image

%load_ext autoreload
%autoreload 2

### Sample data for development

In [3]:
#TRAINING_IMAGES_DIR = './tiny-imagenet-200/train/'
#dev_images = load_training_images(TRAINING_IMAGES_DIR)
#dev_images = np.load('dev.npy')

In [4]:
#print('x_dev shape:', dev_images.shape)

In [5]:
#np.save('dev', dev_images)

### Preprocessing

In [6]:
#dev_images = dev_images / 255.0
#mean = np.array(np.mean(dev_images, axis=(0, 1, 2))).reshape([1,1,1,3])
#std = np.array(np.std(dev_images, axis=(0, 1, 2))).reshape([1,1,1,3])
#dev_images = (dev_images-mean) / std

In [7]:
#cov = np.cov(dev_images.reshape([-1,3]).T)
#eigenvalue, eigenvector = np.linalg.eig(cov)

In [8]:
#np.save('./dev/mean', mean)
#np.save('./dev/std', std)
#np.save('./dev/eigenvalue', eigenvalue)
#np.save('./dev/eigenvector', eigenvector)

In [2]:
mean = np.load('./dev/mean.npy')
std = np.load('./dev/std.npy')
eigenvalue = np.load('./dev/eigenvalue.npy')
eigenvector = np.load('./dev/eigenvector.npy')

In [3]:
def ImageNet_preprocess_train(image):
    image = image / 255.0
    image = (image-mean) / std
    image = standard_color_aug(image, eigenvalue, eigenvector)
    # scale and aspect ratio augmentation
    area = image.shape[1] * image.shape[2]
    target_area = np.random.uniform(0.08,1) * area
    aspect_ratio = np.random.uniform(3/4, 4/3)
    w = int(np.floor(np.sqrt(target_area * aspect_ratio)))
    h = int(np.floor(np.sqrt(target_area / aspect_ratio)))
    if w<=64 and h<=64:
        crop_image = tf.image.random_crop(image, [1, w, h, 3])
        crop_image = tf.image.resize(crop_image, [64, 64])
        return crop_image
    return image

def ImageNet_preprocess_test(image):
    image = image / 255.0
    image = (image-mean) / std
    return image

In [4]:
train_datagen = ImageDataGenerator(
            horizontal_flip=True,
            preprocessing_function=ImageNet_preprocess_train,
            validation_split=0.1)
val_datagen = ImageDataGenerator(preprocessing_function=ImageNet_preprocess_test, validation_split=0.1)
test_datagen = ImageDataGenerator(preprocessing_function=ImageNet_preprocess_test)

In [5]:
BATCH_SIZE=64
train_generator = train_datagen.flow_from_directory('./tiny-imagenet-200/train/',target_size=(64, 64), subset='training', seed = 1, batch_size=BATCH_SIZE)
valid_generator = val_datagen.flow_from_directory('./tiny-imagenet-200/train/',target_size=(64, 64), subset='validation', seed = 1, batch_size=BATCH_SIZE)
test_generator = test_datagen.flow_from_directory('./tiny-imagenet-200/val/',target_size=(64, 64), shuffle= False, batch_size=1)

Found 90000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Found 10000 images belonging to 1 classes.


### Modeling

In [4]:
img_input = Input(shape=(64, 64, 3))
output = AttentionResNet56(img_input)
model = Model(img_input, output)

In [23]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_110 (Conv2D)             (None, 64, 64, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_111 (BatchN (None, 64, 64, 64)   256         conv2d_110[0][0]                 
__________________________________________________________________________________________________
activation_114 (Activation)     (None, 64, 64, 64)   0           batch_normalization_111[0][0]    
____________________________________________________________________________________________

Total params: 2,639,720
Trainable params: 2,620,328
Non-trainable params: 19,392
__________________________________________________________________________________________________


In [19]:
model.compile(optimizer = tf.keras.optimizers.SGD(0.001, momentum=0.9, name='SGD', decay=1e-5),
             loss = 'categorical_crossentropy',   #imagegenerator has encoded label
             metrics = 'accuracy')

In [8]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3)

In [6]:
checkpoint_filepath = './save_models/ImageNet_56.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,  #can only save weight now
    monitor='val_accuracy',
    mode='max',
    save_freq='epoch',
    save_best_only=True)

In [20]:
#model.load_weights(checkpoint_filepath)
history = model.fit(train_generator,  validation_data=valid_generator, epochs = 20, callbacks=[ early_stopping_cb, model_checkpoint_callback]) 

Epoch 1/20
1407/1407 [==============================] - 487s 346ms/step - loss: 5.2153 - accuracy: 0.0240 - val_loss: 5.0847 - val_accuracy: 0.0360
Epoch 2/20
1407/1407 [==============================] - 486s 345ms/step - loss: 4.8469 - accuracy: 0.0456 - val_loss: 4.6795 - val_accuracy: 0.0596
Epoch 3/20
1407/1407 [==============================] - 486s 346ms/step - loss: 4.6744 - accuracy: 0.0633 - val_loss: 4.6178 - val_accuracy: 0.0783
Epoch 4/20
1407/1407 [==============================] - 487s 346ms/step - loss: 4.5325 - accuracy: 0.0788 - val_loss: 4.5335 - val_accuracy: 0.0847
Epoch 5/20
1407/1407 [==============================] - 485s 345ms/step - loss: 4.4176 - accuracy: 0.0941 - val_loss: 4.5139 - val_accuracy: 0.1066
Epoch 6/20
 408/1407 [=======>......................] - ETA: 5:36 - loss: 4.3456 - accuracy: 0.1043

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1407/1407 [==============================] - 485s 345ms/step - loss: 4.3064 - accuracy: 0.1083 - val_loss: 4.1256 - val_accuracy: 0.1303
Epoch 7/20
1407/1407 [==============================] - 486s 345ms/step - loss: 4.2074 - accuracy: 0.1206 - val_loss: 4.0122 - val_accuracy: 0.1455
Epoch 8/20
1407/1407 [==============================] - 486s 346ms/step - loss: 4.1141 - accuracy: 0.1347 - val_loss: 3.8445 - val_accuracy: 0.1725
Epoch 9/20
1407/1407 [==============================] - 485s 345ms/step - loss: 4.0232 - accuracy: 0.1476 - val_loss: 4.2586 - val_accuracy: 0.1671
Epoch 10/20
1407/1407 [==============================] - 487s 346ms/step - loss: 3.9526 - accuracy: 0.1572 - val_loss: 3.8144 - val_accuracy: 0.1790
Epoch 11/20
1407/1407 [==============================] - 487s 346ms/step - loss: 3.8909 - accuracy: 0.1679 - val_loss: 3.7365 - val_accuracy: 0.2044
Epoch 12/20
1407/1407 [==============================] - 486s 345ms/step - loss: 3.8168 - accuracy: 0.1777 - val_loss: 3.

In [21]:
history = model.fit(train_generator,  validation_data=valid_generator, epochs = 20, callbacks=[ early_stopping_cb, model_checkpoint_callback]) 

Epoch 1/20
1407/1407 [==============================] - 487s 346ms/step - loss: 3.3885 - accuracy: 0.2504 - val_loss: 3.2079 - val_accuracy: 0.2778
Epoch 2/20
1407/1407 [==============================] - 487s 346ms/step - loss: 3.3504 - accuracy: 0.2561 - val_loss: 3.1583 - val_accuracy: 0.2908
Epoch 3/20
1407/1407 [==============================] - 484s 344ms/step - loss: 3.3132 - accuracy: 0.2637 - val_loss: 3.1903 - val_accuracy: 0.2899
Epoch 4/20
1407/1407 [==============================] - 487s 346ms/step - loss: 3.2705 - accuracy: 0.2693 - val_loss: 3.0883 - val_accuracy: 0.3039
Epoch 5/20
1407/1407 [==============================] - 486s 345ms/step - loss: 3.2397 - accuracy: 0.2779 - val_loss: 3.1586 - val_accuracy: 0.2960
Epoch 6/20
1407/1407 [==============================] - 487s 346ms/step - loss: 3.2148 - accuracy: 0.2794 - val_loss: 3.0352 - val_accuracy: 0.3134
Epoch 7/20
1407/1407 [==============================] - 486s 345ms/step - loss: 3.1823 - accuracy: 0.2874 - val_

In [22]:
history = model.fit(train_generator,  validation_data=valid_generator, epochs = 20, callbacks=[ early_stopping_cb, model_checkpoint_callback]) 

Epoch 1/20
1407/1407 [==============================] - 484s 344ms/step - loss: 2.8568 - accuracy: 0.3443 - val_loss: 2.8492 - val_accuracy: 0.3610
Epoch 2/20
1407/1407 [==============================] - 484s 344ms/step - loss: 2.8280 - accuracy: 0.3494 - val_loss: 2.8322 - val_accuracy: 0.3549
Epoch 3/20
1407/1407 [==============================] - 484s 344ms/step - loss: 2.8100 - accuracy: 0.3538 - val_loss: 2.8811 - val_accuracy: 0.3579
Epoch 4/20
1407/1407 [==============================] - 484s 344ms/step - loss: 2.7978 - accuracy: 0.3566 - val_loss: 2.8449 - val_accuracy: 0.3588
Epoch 5/20
1407/1407 [==============================] - 483s 343ms/step - loss: 2.7671 - accuracy: 0.3614 - val_loss: 2.8518 - val_accuracy: 0.3605


In [8]:
model.save_weights('./save_models/ImageNet_56.h5')

### Predict

In [23]:
# Match the class names and labels
test_label = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'], usecols=['File', 'Class'])
class_indices = pd.Series(train_generator.class_indices, name='label')
test_label = pd.merge(test_label, class_indices, how='left', left_on = 'Class', right_index=True).set_index('File').drop('Class',axis=1)

In [24]:
test_names= os.listdir('./tiny-imagenet-200/val/images/')
test_names.sort() # Test data generator sorts the data in alphanumeric order
index_order = pd.Series(test_names, name='File')

In [25]:
test_label = test_label.reindex(index=index_order)

In [26]:
NUM_TEST = 2000
y_test = test_label[:NUM_TEST]

In [27]:
test_generator = test_datagen.flow_from_directory('./tiny-imagenet-200/val/',target_size=(64, 64), shuffle= False, batch_size=1)
preds, top_5_preds = predict_10_crop(test_generator, model, NUM_TEST)

Found 10000 images belonging to 1 classes.


In [ ]:
#np.save('preds.npy',preds)
#np.save('top_5_preds.npy',top_5_preds)

In [28]:
y_test.loc[:,'predict'] = preds.copy()

/home/ecbm4040/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/ecbm4040/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [29]:
top_1_error = np.mean(y_test['label'] != y_test['predict'])
top_5_error = np.mean(tf.math.in_top_k(y_test['label'][:NUM_TEST].values, top_5_preds, 5))

In [30]:
print('top_1_error:',top_1_error)
print('top_5_error:',top_5_error)

top_1_error: 0.5875
top_5_error: 0.019
